<h1><center>CS 455/595a: MLP Demo using TensorFlow</center></h1>
<center>Richard S. Stansbury</center>

This notebook applies the ANN techniques for the Titanic Survivors and Boston Housing Prediction models covered in [1] with the [Titanic](https://www.kaggle.com/c/titanic/) and [Boston Housing](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html) data sets for DT-based classification and regression, respectively.

Several different approaches to model construction are shown ihe demos below

Reference:

[1] Aurelen Geron. *Hands on Machine Learning with Scikit-Learn & TensorFlow* O'Reilley Media Inc, 2017.

[2] Aurelen Geron. "ageron/handson-ml: A series of Jupyter notebooks that walk you through the fundamentals of Machine Learning and Deep Learning in python using Scikit-Learn and TensorFlow." Github.com, online at: https://github.com/ageron/handson-ml [last accessed 2019-03-01]

**Table of Contents**
1. [Titanic Survivor ANN Classifiers](#Titanic-Survivor-Classifier)
 
2. [Boston Housing Cost Ensemble ANN Regressor](#Boston-Housing-Cost-Estimator)

# Titanic Survivor Classifier

## Set up - Imports of libraries and Data Preparation

In [7]:
from matplotlib import pyplot as plt
%matplotlib inline 
import numpy as np
import pandas as pd
import os
import tensorflow.keras as keras

# From: https://github.com/ageron/handson-ml/blob/master/09_up_and_running_with_tensorflow.ipynb    
def reset_graph():
    keras.backend.clear_session() 
    
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "titanic-logs"
logdir = "{}/run-{}/".format(root_logdir, now)

Import the data and apply pipelines to pre-process the data.

In [8]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.model_selection import train_test_split

# Read data from input files into Pandas data frames
data_path = os.path.join("datasets","titanic")
train_filename = "train.csv"
test_filename = "test.csv"

def read_csv(data_path, filename):
    joined_path = os.path.join(data_path, filename)
    return pd.read_csv(joined_path)

# Read CSV file into Pandas Dataframes
train_df = read_csv(data_path, train_filename)

# Defining Data Pre-Processing Pipelines
class DataFrameSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, attributes):
        self.attributes = attributes
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.attributes]

class MostFrequentImputer(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        self.most_frequent = pd.Series([X[c].value_counts().index[0] for c in X], 
                                       index = X.columns)
        return self
    
    def transform(self, X):
        return X.fillna(self.most_frequent)


numeric_pipe = Pipeline([
        ("Select", DataFrameSelector(["Age", "Fare", "SibSp", "Parch"])), # Selects Fields from dataframe
        ("Imputer", SimpleImputer(strategy="median")),   # Fills in NaN w/ median value for its column
        ("Scaler", StandardScaler()),
    ])

categories_pipe = Pipeline([
        ("Select", DataFrameSelector(["Pclass", "Sex"])), # Selects Fields from dataframe
        ("MostFreqImp", MostFrequentImputer()), # Fill in NaN with most frequent
        ("OneHot", OneHotEncoder(sparse=False, categories='auto')), # Onehot encode
    ])

preprocessing_pipe = FeatureUnion(transformer_list = [
        ("numeric pipeline", numeric_pipe), 
        ("categories pipeline", categories_pipe)
     ]) 

# Process Input Data Using Pipleines
X_data = preprocessing_pipe.fit_transform(train_df)
y_data = train_df["Survived"].values.reshape(-1,1)

# Process the output data.
feature_names = ["Age", "Fare", "SibSp", "Parch", "Class0", "class1","Sex0", "Sex1"]

print(X_data.shape)
print(y_data.shape)

(891, 9)
(891, 1)


Split the data into a training and validation set.

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.33)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(596, 9) (596, 1) (295, 9) (295, 1)


Implementation of the TF.Estimator.DNNClassifier (formerly of TFLearn)

In [10]:
# Construction Phase

import tensorflow as tf

reset_graph()

feature_cols = [tf.feature_column.numeric_column("X", shape=[X_data.shape[1]])]

dnn_clf = tf.estimator.DNNClassifier(hidden_units=[20,20], n_classes=2,
                                     feature_columns=feature_cols)

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": X_train}, y=y_train, batch_size=50, num_epochs=400, shuffle=True)
dnn_clf.train(input_fn=train_input_fn)

test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": X_test}, y=y_test, shuffle=False)

print(X_train.shape)

eval_results = dnn_clf.evaluate(input_fn=test_input_fn)
                                
eval_results


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\RICHAR~1.STA\\AppData\\Local\\Temp\\tmpf5m72o46', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


AttributeError: module 'tensorflow_core.estimator' has no attribute 'inputs'

Now, let's use plain TensorFlow to implement a neural network using the tf.layers.dense class to define fully-connected (dense) layers of RELU and a softmax of the final output.

In [ ]:
reset_graph()

def get_batch(X, iter, size):
    return X[(iter*batch_size) : ((iter+1)*batch_size)]


learning_rate = 0.1

num_features = X_train.shape[1]
num_instances = X_train.shape[0]

# Construction
X = tf.placeholder(tf.float32, shape=(None, num_features), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("Titanic_MLP"):
    hidden1 = tf.layers.dense(X, 20, name="Hidden-1", activation = tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, 10, name="Hidden-2", activation=tf.nn.relu)
    hidden3 = tf.layers.dense(hidden2, 5, name="Hidden-3", activation=tf.nn.relu)
    logits = tf.layers.dense(hidden3, 2, name="Survived")
    output = tf.nn.softmax(logits)
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    loss_summary = tf.summary.scalar('Loss', loss)
    file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

with tf.name_scope("train"): 
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    accuracy_summary = tf.summary.scalar('accuracy', accuracy)
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 100
batch_size = 50


# Execution
with tf.Session() as sess:
    init.run()
    step=0
    
    for epoch in range(n_epochs):
        
        for iteration in range(num_instances // batch_size + 1):
            step+=1
            X_batch = get_batch(X_train, iteration, batch_size)
            y_batch = get_batch(y_train, iteration, batch_size)
            
            sess.run(training_op, feed_dict={X: X_batch,
                                            y: y_batch.reshape(y_batch.shape[0])})
            
        # Save trained model at end of epoch
        save_path = saver.save(sess, "./tf_plain_class.chkp")
        
        #Evaluate on final training batch vs. final testing
        acc_train, loss_summary_str, acc_summary_str = sess.run([accuracy, loss_summary, accuracy_summary],feed_dict={X: X_batch, y: y_batch.reshape(y_batch.shape[0])})
        acc_val = accuracy.eval(feed_dict={X:X_test, y: y_test.reshape(y_test.shape[0])})

        # Logging for Tensor Board for each epoch
        file_writer.add_summary(loss_summary_str,step)
        file_writer.add_summary(acc_summary_str,step)
        
        # Print progress made
        print("{}-Train: {} Test:{}".format(epoch,
                                           acc_train,
                                           acc_val)) 
file_writer.close()

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score

with tf.Session() as sess:
    
    saver.restore(sess, "./tf_plain_class.chkp")
    y_predict = output.eval(feed_dict={X:X_test, y: y_test.reshape(y_test.shape[0])})
    
    con_mx = confusion_matrix(y_test.reshape(1,-1)[0], np.argmax(y_predict, axis=1))
    precision = precision_score(y_test.reshape(1,-1)[0], np.argmax(y_predict, axis=1))
    recall = recall_score(y_test.reshape(1,-1)[0], np.argmax(y_predict, axis=1))
    print("Precision: {}, \nRecall: {}, \nConf Mat: {}". format(precision, recall, con_mx))

In [ ]:
reset_graph()

encoder = OneHotEncoder(sparse=False, categories='auto')

saver_callback = tf.keras.callbacks.ModelCheckpoint("./keras_model_class.ckpt", 
                                                 save_weights_only=True,
                                                 verbose=1)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(20, activation='relu', input_shape=(9,)))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(5, activation='relu'))
model.add(tf.keras.layers.Dense(2, activation='softmax'))

model.compile(optimizer = tf.train.GradientDescentOptimizer(0.1),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

init = tf.global_variables_initializer()

# Execution
with tf.Session() as sess:
    init.run()
    accuracy = model.fit(X_train, y_train, 
              epochs=400, 
              batch_size=100, 
              callbacks = [saver_callback],
              validation_data = (X_test, y_test))

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score

with tf.Session() as sess:
    
    model.load_weights("./keras_model_class.ckpt")
    y_predict = model.predict(X_test)
    
    con_mx = confusion_matrix(y_test.reshape(1,-1)[0], np.argmax(y_predict, axis=1))
    precision = precision_score(y_test.reshape(1,-1)[0], np.argmax(y_predict, axis=1))
    recall = recall_score(y_test.reshape(1,-1)[0], np.argmax(y_predict, axis=1))
    print("Precision: {}, \nRecall: {}, \nConf Mat: {}". format(precision, recall, con_mx))

# Boston Housing Cost Estimator

Building off the classifier examples above, this section shows ensemble regressors using bagging and random forests.

## Setup

In [ ]:
# Load Data Set
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler
boston_housing_data = datasets.load_boston()

scaler = MinMaxScaler()
bouston_housing_data_instances = scaler.fit_transform(boston_housing_data.data)
bouston_housing_data_instances.shape


In [ ]:
reset_graph()

train_X, test_X, train_y, test_y = train_test_split(bouston_housing_data_instances,
                                                   boston_housing_data.target,
                                                   test_size=0.20)

def get_batch(X, iter, size):
    return X[(iter*batch_size) : ((iter+1)*batch_size)]

learning_rate = 0.001

num_features = train_X.shape[1]
num_instances = train_y.shape[0]

# Construction
X = tf.placeholder(tf.float32, shape=(None, num_features), name="X")
y = tf.placeholder(tf.float32, shape=(None), name="y")

with tf.name_scope("Boston-MLP"):
    hidden1 = tf.layers.dense(X, 5, name="Hidden-1", activation = tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, 5, name="Hidden-2", activation = tf.nn.relu)
    output = tf.transpose(tf.layers.dense(hidden2, 1, name="Price"))
    
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.square(y-output))
    
with tf.name_scope("train"): 
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 400
batch_size = 10

# Execution
with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        
        for iteration in range(num_instances // batch_size + 1):
            X_batch = get_batch(train_X, iteration, batch_size)
            y_batch = get_batch(train_y, iteration, batch_size)   
 
            sess.run(training_op, feed_dict={X: X_batch,
                                            y: y_batch})
            

        mse_train = loss.eval(feed_dict={X: X_batch,
                                            y: y_batch})
        mse_val = loss.eval(feed_dict={X:test_X, y: test_y})
        
        print("{}-Train: {} Test:{}".format(epoch,
                                           mse_train,
                                           mse_val))

From the above example, you will see that the model converges early. 

In [ ]:
reset_graph()


train_X, test_X, train_y, test_y = train_test_split(bouston_housing_data_instances,
                                                   boston_housing_data.target,
                                                   test_size=0.33)


model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10, activation='relu', input_shape=(13,)))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(1))


model.compile(optimizer = tf.train.GradientDescentOptimizer(0.001),loss='mean_squared_error',metrics=['mse'])

init = tf.global_variables_initializer()


# Execution
with tf.Session() as sess:
    init.run()
    model.fit(train_X, train_y, epochs=4000, batch_size=200, validation_data = (test_X, test_y))